# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122836e+02     1.414590e+00
 * time: 0.5619759559631348
     1     1.120988e+01     1.043680e+00
 * time: 2.042696952819824
     2    -1.207646e+01     1.008341e+00
 * time: 2.127655029296875
     3    -3.434096e+01     7.119857e-01
 * time: 2.2479920387268066
     4    -4.802453e+01     5.042651e-01
 * time: 2.36666202545166
     5    -5.696970e+01     2.149420e-01
 * time: 2.4969139099121094
     6    -5.976902e+01     1.110563e-01
 * time: 2.571697950363159
     7    -6.085833e+01     5.860489e-02
 * time: 2.647753953933716
     8    -6.129375e+01     7.291428e-02
 * time: 2.7255449295043945
     9    -6.162831e+01     3.518771e-02
 * time: 2.825071096420288
    10    -6.185061e+01     2.515122e-02
 * time: 2.902782917022705
    11    -6.201180e+01     2.120545e-02
 * time: 2.9773740768432617
    12    -6.207090e+01     1.734512e-02
 * time: 3.0542490482330322
    13    -6.212403e+01     1.278108e-02
 * time: 3.1290369033813477
 

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671077
    AtomicLocal         -18.8557720
    AtomicNonlocal      14.8522670
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485385 
    Xc                  -19.3336825

    total               -62.261666461870